In [1]:
from tool.functions import *
from tool.eval2 import *

In [2]:
def get_weight():
    weights = {}
    with open('./files/stats.csv', 'r', encoding = 'utf-8') as inp:
        for line in inp:
            data = line.split()
            weights[data[0]+'-'+data[1]] = 1/log10(10 + float(data[2]) + 0.5 * float(data[3]))
            weights[data[1]+'-'+data[0]] = 1/log10(10 + float(data[2]) + 0.5 * float(data[4]))
    return weights

In [26]:
def weighted_path_length(path):
    global weights
    length = 0
    for key, value in enumerate(path[1:]):
        length += weights[path[key].lang+'-'+value.lang]
    #print (length)
    return exp(-length)

In [4]:
def metric(G, word, translation, cutoff, mode='exp'):
    coef = 0
    if mode in ('exp', 'len'):
        t = Counter([len(i) for i in nx.all_simple_paths(G, word, translation, cutoff=cutoff)])
        if mode == 'exp': 
            for i in t: 
                coef += exp(-i)*t[i]
            return coef
        if mode == 'len':
            for i in t: 
                coef += t[i]*i
            return coef
    if mode in ('exp-len'):
        for path in nx.all_simple_paths(G, word, translation, cutoff=5):
            coef += weighted_path_length(path)

In [5]:
weights = get_weight()

In [6]:
list(weights.items())[-10:]

[('vie-eng', 0.6450662562255824),
 ('eng-vie', 0.6910095361682882),
 ('vro-est', 0.44885585875131156),
 ('est-vro', 0.44885585875131156),
 ('zho-spa', 0.24110064881289556),
 ('spa-zho', 0.2411024458388548),
 ('zul-ssw', 0.9266284080291267),
 ('ssw-zul', 0.9266284080291267),
 ('zul-xho', 0.8725028695491559),
 ('xho-zul', 0.8725028695491559)]

In [11]:
lang1, lang2, n = 'eng','spa', 10
%time get_relevant_languages(lang1, lang2)
%time load_file(lang1, lang2, n=n)
%time change_encoding('{}-{}'.format(lang1,lang2))
%time G = built_from_file('{}-{}'.format(lang1,lang2))
%time l1, l2 = dictionaries(lang1, lang2)

Wall time: 694 ms
Wall time: 728 ms
Wall time: 712 ms
Wall time: 1min 24s
Wall time: 6.37 s


In [15]:
metric

<function __main__.metric(G, word, translation, cutoff, mode='exp')>

In [ ]:
%time print_lemma_results(lemma_search (G, 'star', l1, 'spa', 4, 40), n = 5)

In [3]:
lang1, lang2, n = 'eng','spa', 10

In [4]:
%time eval_loop(lang1, lang2, n=10, cutoff=4, n_iter=3, topn=5)#, metric='exp-len')

Precision : 0.9574759945130316, recall : 0.698, f1-score : 0.8074031231925969
Precision : 0.9486166007905138, recall : 0.72, f1-score : 0.8186469584991471
Precision : 0.9504021447721179, recall : 0.709, f1-score : 0.8121420389461625

Wall time: 7min 4s


In [12]:
%time s = get_evaluation_pairs(G, l1, lang2, n=1000)

Wall time: 918 ms


In [13]:
u, v = s[0][0], s[0][1]
u,v

(eng$treasure$[n_n-ND], spa$tesoro$[n_n-m_n-m-ND])

In [27]:
coef = 0
for path in nx.all_simple_paths(G, u, v, cutoff=5):
    coef += weighted_path_length(path)

In [25]:
coef

40.8094240253911